* SCB_37_Dash_Inspector_Notebook - verkar inte vara problem

* Issue [37](https://github.com/salgo60/SCB-Wikidata/issues/37)
* denna Notebook [SCB_37_Dash_Inspector_Notebook.ipynb](https://github.com/salgo60/SCB-Wikidata/blob/main/notebook/SCB_37_Dash_Inspector_Notebook.ipynb)

In [1]:
import time

from datetime import datetime

now = datetime.now()
timestamp = now.timestamp()

start_time = time.time()
print("Start:", datetime.now().strftime("%Y-%m-%d %H:%M:%S"))

Start: 2025-11-20 13:43:04


In [5]:
# Notebook: Dash Inspector for Myndighetsregistret
# Läs in ALLA exporterade filer i underkatalogen "myndighetsreg/"
# Undersök vilka typer av bindestreck som förekommer i organisationsnummer
# Gruppera poster efter vilket Unicode-dash-tecken som används

import pandas as pd
import os
import re
from collections import defaultdict

# --- 1. Hitta alla filer i katalogen ---
DIR = 'myndighetsreg_export'
files = [os.path.join(DIR, f) for f in os.listdir(DIR) if f.lower().endswith('.txt')]

print("Hittade filer:")
for f in files:
    print(f)
# --- 2. Funktion: identifiera vilken typ av dash som används ---
DASH_MAP = {
    '-': 'ASCII hyphen (U+002D)',
    '\u2013': 'EN DASH (U+2013)',
    '\u2014': 'EM DASH (U+2014)',
    '\u2212': 'MINUS SIGN (U+2212)',
    '\u2010': 'HYPHEN (U+2010)',
    '\u2011': 'NON-BREAKING HYPHEN (U+2011)',
    '\ufe58': 'SMALL EM DASH (U+FE58)',
    '\u2043': 'HYPHEN BULLET (U+2043)'
}

ALL_DASHES = set(DASH_MAP.keys())


def find_dash_type(orgnr):
    """Returnerar en lista med vilka dash-tecken som förekommer i strängen."""
    if pd.isna(orgnr):
        return []

    s = str(orgnr)
    found = []
    for ch in s:
        if ch in ALL_DASHES:
            found.append(ch)
    return found

# --- 3. Läs in alla filer och samla dash-typer ---
groups = defaultdict(list)  # key = dash type, value = list of rows

for path in files:
    try:
        df = pd.read_csv(path, sep='\t', dtype=str, encoding='utf-8')
    except Exception as e:
        print('Fel vid läsning av', path, e)
        continue

    col = None
    for c in df.columns:
        if 'org' in c.lower():  # heuristik
            col = c
            break

    if not col:
        print('Ingen organisationsnummerkolumn hittad i', path)
        continue

    for i, row in df.iterrows():
        org = row[col]
        dash_types = find_dash_type(org)
        if not dash_types:
            groups['INGET DASH'].append((path, org))
        else:
            for d in dash_types:
                groups[DASH_MAP.get(d, f'UNKNOWN ({d})')].append((path, org))

# --- 4. Sammanställning ---
print("\n=== Sammanfattning av dash-typer ===")
for dash, rows in groups.items():
    print(f"{dash}: {len(rows)} poster")



Hittade filer:
myndighetsreg_export/Statliga förvaltningsmyndigheter.txt
myndighetsreg_export/Svenska utlandsmyndigheter.txt
myndighetsreg_export/Statliga affärsverk.txt
myndighetsreg_export/Sveriges domstolar samt Domstolsverket.txt
myndighetsreg_export/Myndigheter under riksdagen.txt
Ingen organisationsnummerkolumn hittad i myndighetsreg_export/Svenska utlandsmyndigheter.txt

=== Sammanfattning av dash-typer ===
ASCII hyphen (U+002D): 341 poster


### Skapa en fil för hela database

In [6]:
import os
import pandas as pd

DIR = "Myndighetsreg_export"

# Hitta alla txt-filer
files = [os.path.join(DIR, f) for f in os.listdir(DIR) if f.lower().endswith('.txt')]

dfs = []

for file in files:
    # Justera sep beroende på filtyp: '\t', ';', ',' osv.
    df = pd.read_csv(file, sep="\t", encoding="utf-8")  
    dfs.append(df)

# Slå ihop till en enda DataFrame
merged = pd.concat(dfs, ignore_index=True)

# Spara som en CSV
merged.to_csv("Myndighetsreg_concat/Myndighetsreg_alla.csv", index=False, encoding="utf-8")

print("Klart! skapade sammanstallt.csv")


Klart! skapade sammanstallt.csv


In [18]:
import pandas as pd
import requests
from tqdm.notebook import tqdm

tqdm.pandas()   # aktiverar progress_apply()

def check_url(url):
    """Returnera HTTP-statuskod (eller None) och OK/Fail."""
    try:
        checkurl = "http://" + url.lstrip("/")
        r = requests.head(checkurl, allow_redirects=True, timeout=5)
        return r.status_code, (r.status_code < 400)
    except Exception:
        return None, False

# Progressbar visas automatiskt
merged["HTTP_Status"], merged["URL_OK"] = zip(*merged["Webbadress"].progress_apply(check_url))

# Filtrera trasiga länkar
broken = merged[~merged["URL_OK"]]

print("Antal trasiga länkar:", len(broken))

# Lista trasiga url:er
print("\n📌 Felaktiga URL:er:")
for url in broken["Webbadress"]:
    print(url)

# Spara rapport
broken.to_csv("trasiga_webbadresser.csv", index=False)

display(broken)


  0%|          | 0/455 [00:00<?, ?it/s]

Antal trasiga länkar: 54

📌 Felaktiga URL:er:
www.bolagsverket.se
www.csn.se
nan
www.domstol.se/domarnamnden
www.fmv.se
www.fra.se
www.forsvarsmakten.se
nan
www.hb.se
www.imy.se
www.kommerskollegium.se
www.konkurrensverket.se
http://www.regeringen.se/myndigheter-med-flera/krigsforsakringsnamnden/
www.kriminalvarden.se
www.osthammar.se/sv/organisation/namnd/lokala-sakerhetsnamnden/
www.oskarshamn.se/mer-om-kommunen/karnenergi-och-karnavfall/lokala-sakerhetsnamnden-vid-oskarshamns-karnkraftverk-och-karnenergiberedskap/
www.mtfa.se
www.domstol.se/Om-Sveriges-Domstolar/Notarienamnden
https://npof.se
www.bolagsverket.se/om/oss/fler/patentombud
www.raa.se
www.riksarkivet.se
nan
www.rymdstyrelsen.se
www.domstol.se/rattshjalpsnamnden
www.sida.se
www.kammarkollegiet.se/kammarkollegiet/vi-arbetar-ocksa-med/naemndmyndigheter/skiljenaemnden-i-vissa-trygghetsfragor
www.lotteriinspektionen.se
www.sfhm.se
http://www.kammarkollegiet.se/n-mndmyndigheter/skaderegleringsn-mnden
www.kammarkollegiet.se/nam

,Organisationsnr,Namn,PostAdress,PostNr,PostOrt,BesöksAdress,BesöksPostNr,BesöksPostOrt,Tfn,Fax,...,BesöksAdress1,BesöksAdress2,BesöksAdress3,BesöksAdress4,BesöksAdress5,BesöksAdress6,BesöksAdress7,Sort,URL_OK,HTTP_Status
10,202100-5489,BOLAGSVERKET,NaN,851 81,SUNDSVALL,STUVARVÄGEN 21,852 29,SUNDSVALL,771670670.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN
15,202100-1819,CENTRALA STUDIESTÖDSNÄMNDEN,NaN,851 82,SUNDSVALL,NORRA TJÄRNGATAN 2,852 33,SUNDSVALL,60186000.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN
16,202100-6842,Delegationen för folkrättslig granskning av va...,NaN,103 33,STOCKHOLM,NaN,000 00,STOCKHOLM,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN
18,202100-6081,Domarnämnden,BOX 2330,103 18,STOCKHOLM,BIRGER JARLS TORG 5,111 28,STOCKHOLM,856166950.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN
38,202100-0340,FÖRSVARETS MATERIELVERK,NaN,115 88,STOCKHOLM,BANÉRGATAN 62,115 53,STOCKHOLM,87824000.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,403.0
39,202100-0365,FÖRSVARETS RADIOANSTALT,BOX 301,161 26,BROMMA,RÖRBYVÄGEN 20,178 93,DROTTNINGHOLM,105574600.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,405.0
41,202100-4615,FÖRSVARSMAKTEN,NaN,107 85,STOCKHOLM,Lidingövägen 24,115 57,STOCKHOLM,87887500.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,403.0
44,202100-6859,Granskningsnämnden för försvarsuppfinningar,NaN,103 33,STOCKHOLM,NaN,000 00,STOCKHOLM,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN
51,202100-3138,HÖGSKOLAN I BORÅS,NaN,501 90,BORÅS,ALLÉGATAN 1,503 32,BORÅS,334354000.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,404.0
67,202100-0050,Integritetsskyddsmyndigheten,BOX 8114,104 20,STOCKHOLM,Fleminggatan 14,112 26,STOCKHOLM,86576100.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,404.0


In [ ]:
 # End timer and calculate duration
end_time = time.time()
elapsed_time = end_time - start_time# Bygg audit-lager för den här etappen

# Print current date and total time
print("Date:", datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
minutes, seconds = divmod(elapsed_time, 60)
print("Total time elapsed: {:02.0f} minutes {:05.2f} seconds".format(minutes, seconds))
